# Production constraints by timeslice

In some sectors it may be the case that a technology can only output a certain amount at a certain time. For instance, solar photovoltaics (PV) don't produce power in the dark, and thus their output is limited at night. 

In this section, we explain how to add constraints to outputs of technologies at certain timeslices. This could either by a maximum constraint, for instance with the solar PV example previously mentioned. Or, this could be a minimum constraint, where we expect a minimum amount of output by a nuclear power plant at all times.

## Minimum timeslice

In this tutorial we will be amending the `default_timeslice` example.

Copy this model so you can edit the files:

```bash
python -m muse --model default_timeslice --copy PATH/TO/COPY/THE/MODEL/TO
```

You will see that, compared to the `default` example, this model has an additional `TechnodataTimeslices.csv` file in the `power` and `residential` sectors, which has the columns `ProcessName`, `RegionName`, `Time`, `month`, `day`, `hour`, `UtilizationFactor`, `MinimumServiceFactor`.
The majority of these columns are self explanatory, and correspond to the columns in other `csv` files - for instance, `ProcessName`, `RegionName` and `Time`. 
The `UtilizationFactor` column specifies the maximum utilization factor for the respective technologies in the respective timeslices, and the `MinimumServiceFactor` specifies the minimum service factor of a technology.
The timeslice based columns, however, are dynamic and will match the levels as defined in the `toml` file. 

We will modify the minimum service factor for `gasCCGT` in the `power` sector as follows.

|ProcessName|RegionName|Time|month|day|hour|UtilizationFactor|MinimumServiceFactor|
|-|-|-|-|-|-|-|-|
|Unit|-|Year|-|-|-|-|-|
|gasCCGT|R1|2020|all-year|all-week|night|1|**0.2**|
|gasCCGT|R1|2020|all-year|all-week|morning|1|**0.4**|
|gasCCGT|R1|2020|all-year|all-week|afternoon|1|**0.6**|
|gasCCGT|R1|2020|all-year|all-week|early-peak|1|**0.4**|
|gasCCGT|R1|2020|all-year|all-week|late-peak|1|**0.8**|
|gasCCGT|R1|2020|all-year|all-week|evening|1|**1**|
|windturbine|R1|2020|all-year|all-week|night|1|1|
|windturbine|R1|2020|all-year|all-week|morning|1|1|
|windturbine|R1|2020|all-year|all-week|afternoon|1|1|
|windturbine|R1|2020|all-year|all-week|early-peak|1|1|
|windturbine|R1|2020|all-year|all-week|late-peak|1|1|
|windturbine|R1|2020|all-year|all-week|evening|1|1|


Looking at `settings.toml` file, you should see that the file has already been linked to the appropriate sector:

```toml
[sectors.power]
type = 'default'
priority = 2
dispatch_production = 'costed'
technodata = '{path}/technodata/power/Technodata.csv'
commodities_in = '{path}/technodata/power/CommIn.csv'
commodities_out = '{path}/technodata/power/CommOut.csv'
technodata_timeslices = '{path}/technodata/power/TechnodataTimeslices.csv'
```
    
Notice the `technodata_timeslices` path in the bottom row of the code above.

Once this has been completed, we are able to run MUSE as before, with the following command:

    python -m muse settings.toml

Next, we will visualise the output of the technologies as before. However, this time we will visualise the supply outputs per technology and per timeslice.

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
power_supply_2020 = pd.read_csv(
    "../tutorial-code/7-min-max-timeslice-constraints/1-min-constraint/Results/Power/Supply/2020.csv"
)
gassCCGT_electricity_supply_2020 = (
    power_supply_2020[
        (power_supply_2020.technology == "gasCCGT")
        & (power_supply_2020.commodity == "electricity")
        & (power_supply_2020.year == 2020)
    ]
    .groupby(["timeslice", "technology"])
    .sum()
    .reset_index()
)

ax = sns.barplot(
    data=gassCCGT_electricity_supply_2020,
    x="timeslice",
    y="supply",
    hue="technology",
)
ax.set_title("Supply versus timeslice for the year 2020")

As expected, for the year 2020, the gasCCGT supplies the electricity mix by the minimum per timeslice. This is as specified in the `TechnodataTimeslices.csv` file. Starting from a supply of 0.2 in the first timeslice and ending with a supply of 1 in the last timeslice. The supply increases linearly apart from during the 3rd timeslice, where it reduces to 0.4.

## Maximum timeslice constraint

Next, we want to ensure that the supply of `windturbine` does not exceed a certain value during the day. This may be because, for example, there is reduced wind during the day. We will, therefore, modify the `TechnodataTimeslices.csv` file by changing the values of `UtilizationFactor` and `MinimumServiceFactor` as shown.

|ProcessName|RegionName|Time|month|day|hour|UtilizationFactor|MinimumServiceFactor|
|-|-|-|-|-|-|-|-|
|Unit|-|Year|-|-|-|-|-|
|gasCCGT|R1|2020|all-year|all-week|night|1|0.2|
|gasCCGT|R1|2020|all-year|all-week|morning|1|0.4|
|gasCCGT|R1|2020|all-year|all-week|afternoon|1|0.6|
|gasCCGT|R1|2020|all-year|all-week|early-peak|1|0.4|
|gasCCGT|R1|2020|all-year|all-week|late-peak|1|0.8|
|gasCCGT|R1|2020|all-year|all-week|evening|1|1|
|windturbine|R1|2020|all-year|all-week|night|1|**0**|
|windturbine|R1|2020|all-year|all-week|morning|**0.5**|**0**|
|windturbine|R1|2020|all-year|all-week|afternoon|**0.5**|**0**|
|windturbine|R1|2020|all-year|all-week|early-peak|1|**0**|
|windturbine|R1|2020|all-year|all-week|late-peak|1|**0**|
|windturbine|R1|2020|all-year|all-week|evening|1|**0**|


Once this has been saved, we can run the model again (`python -m muse settings.toml`). Next, we can visualise our results as before. We should hopefully see a reduction in the output of `windturbine` to 0.5 in the 2nd and 3rd timeslices:

In [ ]:
power_supply_2030 = pd.read_csv(
    "../tutorial-code/7-min-max-timeslice-constraints/2-max-constraint/Results/Power/Supply/2030.csv"
)
windturbine_electricity_supply_2030 = (
    power_supply_2030[
        (power_supply_2030.technology == "windturbine")
        & (power_supply_2030.commodity == "electricity")
        & (power_supply_2030.year == 2030)
    ]
    .groupby(["timeslice", "technology"])
    .sum()
    .reset_index()
)

ax = sns.barplot(
    data=windturbine_electricity_supply_2030,
    x="timeslice",
    y="supply",
    hue="technology",
)
ax.set_title("Supply versus timeslice for the year 2030")

As expected, we can see an enforced reduction in `windturbine` output to 0.5 in the 2nd (1) and 3rd (2) timeslices.

## Next steps

This brings us to the end of the user guide! Using the information explained in this tutorial, or following similar steps, you will be able to create complex scenarios of your choosing.